# Assignment Instructions and Cases

## A. Web Scraping

Pada bagian A ini kita akan melakukan pengambilan beberapa data pada website Tokopedia untuk menganalisis bagaimana penjualan produk seblak di Tokopedia 

In [180]:
'''
=========================================================
Graded Challenge 3

Nama : Jeni Kasturi
Batch : BSD-002

Program ini dibuat untuk membantu menganalisis data dari sebuah website dan untuk 
memantau berapa banyaknya produk yang terjual paling banyak menggunakan web scraping.
========================================================
'''
# import library
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from scipy import stats
import numpy as np

driver = webdriver.Chrome()

In [181]:
# membuat list
Nama_Produk = []
Harga_Produk = []
Penjual = []
Kota_Toko = []
Banyaknya_Terjual = []
Rating_Produk = []

#untuk meloop dari halaman 1 sampai 10
for i in range(1,11):
    url="https://www.tokopedia.com/search?navsource=&page={}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=".format(i)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    mini_box = soup.find_all('div',{"class":"pcv3__container css-1izdl9e"})
    number_box = 0

    # untuk meloop isi/value pada masing-masing kolom
    for one_box in mini_box[:20]:
        number_box += 1
        # ekstrak nama produk
        Nama = one_box.find('div',{"class":"prd_link-product-name css-3um8ox"}).get_text()
        Nama_Produk.append(Nama)

        # ektrak harga produk
        Harga = one_box.find('div',{"class":"prd_link-product-price css-h66vau"}).get_text()
        Harga_Produk.append(Harga)

        # ekstrak penjual produk
        Toko = one_box.find('span',{"class":"prd_link-shop-name css-1kdc32b flip"})
        if Toko is not None:
            Penjual.append(Toko.get_text())
        else:
            Penjual.append(None)

        # ekstrak kota dari penjual produk
        Kota = one_box.find('span',{"class":"prd_link-shop-loc css-1kdc32b flip"})
        if Kota is not None:
            Kota_Toko.append(Kota.get_text())
        else:
            Kota_Toko.append(None)

        # ekstrak banyaknya produk terjual
        Terjual = one_box.find('span',{"class":"prd_label-integrity css-1sgek4h"})
        if Terjual is not None:
            Banyaknya_Terjual.append(Terjual.get_text())
        else:
            Banyaknya_Terjual.append(None)

        # ekstrak rating produk terjual
        Rating = one_box.find('span',{"class":"prd_rating-average-text css-t70v7i"})
        if Rating is not None:
            Rating_Produk.append(Rating.get_text())
        else:
            Rating_Produk.append(None)


In [182]:
# untuk menampilkan tabel beserta kolom-kolomnya
df = pd.DataFrame()
df['Nama_Produk'] = Nama_Produk
df['Harga_Produk'] = Harga_Produk
df['Penjual'] = Penjual
df['Kota_Toko'] = Kota_Toko
df['Banyaknya_Terjual'] = Banyaknya_Terjual
df['Rating_Produk'] = Rating_Produk

df 

,Nama_Produk,Harga_Produk,Penjual,Kota_Toko,Banyaknya_Terjual,Rating_Produk
0,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,Rp22.000,None,None,1 rb+ terjual,4.9
1,Seblak Juara ! Seblak Instan Rasa Original - 2...,Rp115.000,None,None,2 rb+ terjual,4.9
2,CUANKIE JUARA! CUANKIE JAJANAN KULINER ENAK KH...,Rp115.000,None,None,500+ terjual,4.9
3,Cuanki / Cuankie Lidah Topping Baso Aci / Sebl...,Rp110.000,None,None,40+ terjual,4.8
4,siomay mini kering gurilem 1kg,Rp23.000,None,None,250+ terjual,4.9
...,...,...,...,...,...,...
178,KERUPUK SEBLAK RAFAEL KERING BANTAT PEDES DAUN...,Rp16.000,AyundaSnack,Bandung,27 terjual,4.9
179,SEBRING KRUPUK KERUPUK SEBLAK KERING PEDAS DAU...,Rp16.000,Aydaa Snack,Surakarta,50+ terjual,4.9
180,GURILEM MINI 1KG Siomay Kering Toping Baso Aci...,Rp29.999,Pedas Gilaa,Surabaya,500+ terjual,4.9
181,SEBRING CAMPUR/ Seblak kering Bumbu kacang/ va...,Rp20.000,KailaAini_collection,Kab. Bandung,30+ terjual,5.0


In [183]:
# untuk melihat value dan type data pada setiap kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Nama_Produk        183 non-null    object
 1   Harga_Produk       183 non-null    object
 2   Penjual            174 non-null    object
 3   Kota_Toko          174 non-null    object
 4   Banyaknya_Terjual  176 non-null    object
 5   Rating_Produk      172 non-null    object
dtypes: object(6)
memory usage: 8.7+ KB


# B. Data Preparation

Pada bagian B ini kita akan melakukan data preparation yang dimana kita akan menghapus beberapa data yang duplikat atau terdapat missing value, agar data yang kita peroleh itu data murni dan setiap kolomnya terisi value.

In [184]:
# untuk menghapus/drop data yang duplikat
df.drop_duplicates(inplace=True)

# tampilkan info tabel di setiap kolomnya
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, 0 to 172
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Nama_Produk        106 non-null    object
 1   Harga_Produk       106 non-null    object
 2   Penjual            97 non-null     object
 3   Kota_Toko          97 non-null     object
 4   Banyaknya_Terjual  99 non-null     object
 5   Rating_Produk      95 non-null     object
dtypes: object(6)
memory usage: 5.8+ KB


In [185]:
# untuk menghapus/drop missing value
df.dropna(inplace=True)

# tampilkan tabelnya kembali
df

,Nama_Produk,Harga_Produk,Penjual,Kota_Toko,Banyaknya_Terjual,Rating_Produk
6,Pasar Keripik Kerupuk Seblak Mawar Mini Pedas ...,Rp12.500,Pasar Keripik,Bandung,29 terjual,5.0
7,Gelifood Combo 3pcs Seblak Instan Kerupuk Mawa...,Rp39.000,Lidigeli,Kab. Garut,10 terjual,5.0
8,siomay mini kering 1kg / gurilem toping seblak...,Rp25.000,CuankiCulametan,Kab. Garut,100+ terjual,4.9
9,Seblak Instan Sehati,Rp13.000,Official Kartika Sari,Bandung,500+ terjual,5.0
10,Cuanki lidah murah kiloan isi 150 toping sebla...,Rp32.500,CuankiCulametan,Kab. Garut,100+ terjual,5.0
...,...,...,...,...,...,...
168,Toping Seblak Baso Aci PAKET MIX 150pcs Cuanki...,Rp77.280,Onesupp Acc,Jakarta Barat,6 terjual,5.0
169,"makaroni kriuk 500g pedas extra daun jeruk,cem...",Rp23.000,anugrah seblak,Kab. Bandung,250+ terjual,4.8
170,PAKET 1 BOCI EXTRA PEDAS + 1 TULANG RANGU + 1 ...,Rp50.000,JM Snacks,Bandung,30+ terjual,4.9
171,Seblak Instan Batagor/Seblak Instan Pedas,Rp8.700,yakoyako,Bekasi,100+ terjual,5.0


In [186]:
# ubah value yang salah menjadi benar
df['Harga_Produk'] = df['Harga_Produk'].str.replace('Rp', '')
df['Harga_Produk'] = df['Harga_Produk'].str.replace('.', '')
df['Banyaknya_Terjual'] = df['Banyaknya_Terjual'].str.replace(' ', '')
df['Banyaknya_Terjual'] = df['Banyaknya_Terjual'].str.replace('rb', '000')
df['Banyaknya_Terjual'] = df['Banyaknya_Terjual'].str.replace('+', '')
df['Banyaknya_Terjual'] = df['Banyaknya_Terjual'].str.replace('terjual', '')

Dari syntax di atas kita bisa lihat bahwa pada tabel seblak kita harus mengganti tipe data pada kolom Harga_Produk, Banyaknya_Terjual, dan Rating_Produk.
Sebelum kita ganti tipe datanya, langkah pertama yang harus kita lakukan yaitu menghapus value-value yang salah menjadi benar, seperti pada kolom Harga_Produk yaitu yang harus kita ubah yaitu lambang (Rp) dan (.) harus kita hilangkan, dan pada kolom Banyaknya_Terjual lambang (rb), (+), (terjual), dan spasinya harus kita hilangkan agar tipe datanya bisa diubah dan komputer bisa menghitung valuenya

In [187]:
# tampilkan tabelnya kembali sebanyak 30 baris teratas
df.head(30)

,Nama_Produk,Harga_Produk,Penjual,Kota_Toko,Banyaknya_Terjual,Rating_Produk
6,Pasar Keripik Kerupuk Seblak Mawar Mini Pedas ...,12500,Pasar Keripik,Bandung,29,5.0
7,Gelifood Combo 3pcs Seblak Instan Kerupuk Mawa...,39000,Lidigeli,Kab. Garut,10,5.0
8,siomay mini kering 1kg / gurilem toping seblak...,25000,CuankiCulametan,Kab. Garut,100,4.9
9,Seblak Instan Sehati,13000,Official Kartika Sari,Bandung,500,5.0
10,Cuanki lidah murah kiloan isi 150 toping sebla...,32500,CuankiCulametan,Kab. Garut,100,5.0
11,Bundle 3pcs - Seblak Basranger Extra Pedas 200...,51900,Rak Sebelah,Jakarta Pusat,50,4.8
12,Seblak Merah Pedas / Kerupuk Bawang Oleh Oleh ...,9000,The Little Snacks,Tangerang Selatan,500,4.9
13,"baso aci boci instan maci eco ayam, keju, seblak",8999,snackbox88,Bekasi,100,5.0
14,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,14900,Lidigeli,Kab. Garut,90,4.7
15,Kerupuk Seblak Pedas Gurih aroma Kencur,15000,DUO BOCIL SNACK,Depok,1000,4.7


In [188]:
# mengganti type data pada beberapa kolom tertentu
df['Harga_Produk']=df['Harga_Produk'].astype(int)
df['Banyaknya_Terjual']=df['Banyaknya_Terjual'].astype(int)
df['Rating_Produk']=df['Rating_Produk'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 6 to 172
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Nama_Produk        86 non-null     object 
 1   Harga_Produk       86 non-null     int32  
 2   Penjual            86 non-null     object 
 3   Kota_Toko          86 non-null     object 
 4   Banyaknya_Terjual  86 non-null     int32  
 5   Rating_Produk      86 non-null     float64
dtypes: float64(1), int32(2), object(3)
memory usage: 4.0+ KB


Setelah kita hapus value-value yang salah, langkah selanjutnya adalah kita ganti tipe data pada kolom Harga_Produk, Banyaknya_Terjual, dan Rating_Produk agar komputer dapat menghitung valuenya, kita sesuaikan tipedatanya dengan valuenya, seperti pada kolom Harga_Produk dan kolom Banyaknya_Terjual kita ubah tipedatanya sebagai integer(int), dan pada kolom Rating_Produk kita ubah tipedatanya menjadi float(float) agar ratingnya dapat terbaca.

# C. Business Understanding/Problem Statement

Pada bagian C ini kita akan menganalisis pernyataan masalah bagaimana meningkatkan pembelian bada produk yang kita jual menggunakan SMART. Untuk lebih jelasnya lihat penjabaran dibawah ini :

### Definisi Pernyataan Masalah - Menggunakan kerangka SMART:

Specific : Target utama pada case ini yaitu ingin menambahkan pendapatan dengan cara menjadi dropship Seblak di Tokopedia. 

Measurable : Pada case ini kita bisa mendapatkan data yang spesifik seperti produk seblak, harga, lokasi toko, review dll.

Achievable : Dalam hal ini web scraping dapat optimal digunakan untuk menganalisis produk seblak yang ada di Tokopedia.

Relevan : Mengetahui potensi pendapatan terendah dan tertinggi yang didapatkan oleh seorang dropship dalam menjual Seblak.

Time-bound : Proses analisis dalam case ini dilakukan dalam waktu 1 minggu untuk mendapatkan data penjualan seblak di Tokopedia.

Problem statement : Dalam hal ini metrik yang digunakan untuk mengetahui jumlah pendapatan setiap produk seblak yang terjual yakni dengan metode web scraping, dimana hal ini membantu dalam proses mengetahui list produk seblak, variasi harga dan tingkat penjualan.

# D. Analysis

Pada bagian D ini kita akan melakukan analisa data untuk mencapai tujuan yang diinginkan, seperti mencari mean, median, modus, standar deviasi, skewness, dan kurtosis dari kolom harga, banyak produk terjual, dan rating. Lalu kita juga harus menampilkan berapa potensi minimum dan maksimum pendapatan menggunakan confidence interval, dan mencaritahu apakah harga barang di Jabodetabek dan di luar Jabodetabek berbeda atau tidak menggunakan uji hipotesis, dan kita caritahu apakah orang-orang cenderung suka dengan produk yang harganya murah atau tidak.

In [189]:
# kolom Harga_Produk

# untuk mencari nilai mean pada tabel Harga_produk
Mean_Harga_Produk = df['Harga_Produk'].mean()
print(f'Nilai rata-rata dari harga produk adalah : {Mean_Harga_Produk}')

# untuk mencari nilai median pada tabel Harga_produk
Median_Harga_Produk = df['Harga_Produk'].median()
print(f'Nilai tengah dari harga produk adalah : {Median_Harga_Produk}')

# untuk mencari nilai modus pada tabel Harga_produk
Modus_Harga_Produk = df['Harga_Produk'].mode()
print(f'Nilai yang sering muncul dari harga produk adalah : {Modus_Harga_Produk}')

# untuk mencari standar deviasi pada tabel Harga_produk
Std_Harga_Produk = df['Harga_Produk'].std()
print(f'Standar deviasi dari harga produk adalah : {Std_Harga_Produk}')

# untuk mencari skew pada kolom Harga_Produk (skew itu berfungsi untuk mencari seberapa simetrisnya bentuk distribui antara mean,meadian,modus)
# skew diatas 0,5 positive skew, dan dibawah -0,5 negative skew
Skew_Harga_Produk = df['Harga_Produk'].skew()
print(f'Skew dari harga produk adalah : {Skew_Harga_Produk}')

# untuk mencari kurtosis pada kolom Harga_Produk (kurtosis itu berfungsi untuk mencari seberapa lancipnya bentuk distribui antara mean,meadian,modus)
# skew diatas 0,5 Leptokurtic, dan dibawah -0,5 Platykurtic
Kurtosis_Harga_produk = df['Harga_Produk'].kurtosis()
print(f'Kurtois dari harga produk adalah : {Kurtosis_Harga_produk}')


Nilai rata-rata dari harga produk adalah : 23231.988372093023
Nilai tengah dari harga produk adalah : 18000.0
Nilai yang sering muncul dari harga produk adalah : 0    15000
Name: Harga_Produk, dtype: int32
Standar deviasi dari harga produk adalah : 22096.03234466165
Skew dari harga produk adalah : 2.9428277780484398
Kurtois dari harga produk adalah : 11.873729522406066


pada syntax diatas kita cari mean, median, modus, standar deviasi, skew, dan kurtosis pada kolom Harga_Produk untuk melihat hasil perhitungannya agar bisa kita tentukan t-stats dan p-valuenya

In [190]:
# kolom Banyaknya_Terjual

# untuk mencari nilai mean pada tabel Banyaknya_Terjual
Mean_Banyaknya_Terjual = df['Banyaknya_Terjual'].mean()
print(f'Nilai rata-rata dari banyaknya terjual produk adalah : {Mean_Banyaknya_Terjual}')

# untuk mencari nilai median pada tabel Banyaknya_Terjual
Median_Banyaknya_Terjual = df['Banyaknya_Terjual'].median()
print(f'Nilai tengah dari banyaknya terjual produk adalah : {Median_Banyaknya_Terjual}')

# untuk mencari nilai modus pada tabel Banyaknya_Terjual
Modus_Banyaknya_Terjual = df['Banyaknya_Terjual'].mode()
print(f'Nilai yang sering muncul dari banyaknya terjual produk adalah : {Modus_Banyaknya_Terjual}')

# untuk mencari standar deviasi pada tabel Banyaknya_Terjual
Std_Banyaknya_Terjual = df['Banyaknya_Terjual'].std()
print(f'Standar deviasi dari banyaknya terjual produk adalah : {Std_Banyaknya_Terjual}')

# untuk mencari skew pada kolom Banyaknya_Terjual
Skew_Banyaknya_Terjual = df['Banyaknya_Terjual'].skew()
print(f'Skew dari banyaknya terjual produk adalah : {Skew_Banyaknya_Terjual}')

# untuk mencari kurtosis pada kolom Banyaknya_Terjual
Kurtosis_Banyaknya_Terjual = df['Banyaknya_Terjual'].kurtosis()
print(f'Kurtois dari banyaknya terjual produk adalah : {Kurtosis_Banyaknya_Terjual}')



# bikin markdown

Nilai rata-rata dari banyaknya terjual produk adalah : 662.0813953488372
Nilai tengah dari banyaknya terjual produk adalah : 90.0
Nilai yang sering muncul dari banyaknya terjual produk adalah : 0    100
Name: Banyaknya_Terjual, dtype: int32
Standar deviasi dari banyaknya terjual produk adalah : 1820.6035858545006
Skew dari banyaknya terjual produk adalah : 4.15480724902674
Kurtois dari banyaknya terjual produk adalah : 17.933798536927483


pada syntax diatas kita cari mean, median, modus, standar deviasi, skew, dan kurtosis pada kolom Banyaknya_Terjual untuk melihat hasil perhitungannya agar bisa kita tentukan t-stats dan p-valuenya

In [191]:
# kolom Rating_Produk

# untuk mencari nilai mean pada tabel Rating_Produk
Mean_Rating_Produk = df['Rating_Produk'].mean()
print(f'Nilai rata-rata dari banyaknya rating produk adalah : {Mean_Rating_Produk}')

# untuk mencari nilai median pada tabel Rating_Produk
Median_Rating_Produk = df['Rating_Produk'].median()
print(f'Nilai tengah dari banyaknya rating produk adalah : {Median_Rating_Produk}')

# untuk mencari nilai modus pada tabel Rating_Produk
Modus_Rating_Produk = df['Rating_Produk'].mode()
print(f'Nilai yang sering muncul dari banyaknya rating produk adalah : {Modus_Rating_Produk}')

# untuk mencari standar deviasi pada tabel Rating_Produk
Std_Rating_Produk = df['Rating_Produk'].std()
print(f'Standar deviasi dari banyaknya rating produk adalah : {Std_Rating_Produk}')

# untuk mencari skew pada kolom Rating_Produk
Skew_Rating_Produk = df['Rating_Produk'].skew()
print(f'Skew dari banyaknya rating produk adalah : {Skew_Rating_Produk}')

# untuk mencari kurtosis pada kolom Rating_Produk
Kurtosis_Rating_Produk = df['Rating_Produk'].kurtosis()
print(f'Kurtois dari banyaknya rating produk adalah : {Kurtosis_Rating_Produk}')



# bikin markdown

Nilai rata-rata dari banyaknya rating produk adalah : 4.901162790697675
Nilai tengah dari banyaknya rating produk adalah : 4.9
Nilai yang sering muncul dari banyaknya rating produk adalah : 0    5.0
Name: Rating_Produk, dtype: float64
Standar deviasi dari banyaknya rating produk adalah : 0.10231920761146476
Skew dari banyaknya rating produk adalah : -0.8332448101631339
Kurtois dari banyaknya rating produk adalah : -0.10962407438427091


pada syntax diatas kita cari mean, median, modus, standar deviasi, skew, dan kurtosis pada kolom Rating_Produk untuk melihat hasil perhitungannya agar bisa kita tentukan t-stats dan p-valuenya

In [192]:
# untuk mencari hasil min dari pendapatan penjualan seblak menggunakan min pada kolom Harga_Produk
Min_Harga_Produk = df['Harga_Produk'].min()
print(f'Nilai min dari hasil pendapatan pada kolom harga produk adalah : {Min_Harga_Produk}')

# untuk mencari hasil max dari pendapatan penjualan seblak menggunakan max pada kolom Harga_Produk
Max_Harga_Produk = df['Harga_Produk'].max()
print(f'Nilai max dari hasil pendapatan pada kolom harga produk adalah : {Max_Harga_Produk}')

# untuk mencari lower dan upper limit pada kolom Harga_Produk
Std_Harga_Produk
N = len(df)

low, up = stats.norm.interval(0.95, loc=df['Harga_Produk'].mean(), scale=Std_Harga_Produk/np.sqrt(N))
print('Lower Limit : ',low)
print('Upper Limit : ',up)

Nilai min dari hasil pendapatan pada kolom harga produk adalah : 1200
Nilai max dari hasil pendapatan pada kolom harga produk adalah : 147500
Lower Limit :  18562.028354026304
Upper Limit :  27901.94839015974


lalu kita cari nilai min dan maxnya dan cari lower dan upper limit dari kolom Harga_Produk untuk mendapatkan batas atas dan batas bawahnya berapa

In [197]:
hasil_low = Mean_Banyaknya_Terjual * low
hasil_up = Mean_Banyaknya_Terjual * up

print("hasil perhitungan dengan penjualan terendah adalah : ", hasil_low)
print("hasil perhitunagn dengan penjualan tertinggi adalah : ", hasil_up)

hasil perhitungan dengan penjualan terendah adalah :  12289573.633138414
hasil perhitunagn dengan penjualan tertinggi adalah :  18473360.9231082


setelah mendapatkan hasil rata-rata banyaknya penjualan produk, langkah selanjutnya kita hitung dengan lower dan upper untuk mendapatkan hasil perhitungan terendah dan tertingginya.

In [194]:
# untuk menentukan apakah harga barang di Jabodetabek dan di luar Jabodetabek berbeda atau tidak

# buat variabel baru yang berisikan nama-nama daerah apa saja yang masuk ke jabodetabek atau tidak
jabodetabek = df[df['Kota_Toko'].isin(['Depok', 'Tangerang Selatan', 'Kab. Tangerang','Jakarta Timur','Jakarta Selatan','Kab. Bogor','Jakarta Barat','Jakarta Pusat','Tangerang','Bekasi','Kab. Bekasi'])][['Harga_Produk', 'Rating_Produk']]
luar_jabodetabek = df[~df['Kota_Toko'].isin(['Depok', 'Tangerang Selatan', 'Kab. Tangerang','Jakarta Timur','Jakarta Selatan','Kab. Bogor','Jakarta Barat','Jakarta Pusat','Tangerang','Bekasi','Kab. Bekasi'])][['Harga_Produk', 'Rating_Produk']]

harga_jabodetabek = jabodetabek['Harga_Produk']
harga_luar_jabodetabek = luar_jabodetabek['Harga_Produk']

print('Rata-rata harga barang di jabodetabek adalah =',harga_jabodetabek.mean())
print('Rata-rata harga barang di luar jabodetabek adalah =',harga_luar_jabodetabek.mean())

Rata-rata harga barang di jabodetabek adalah = 24189.425
Rata-rata harga barang di luar jabodetabek adalah = 22399.434782608696


In [195]:
# buat perhitungan tstat dan p-value
t_stat, p_val = stats.ttest_ind(harga_jabodetabek,harga_luar_jabodetabek)
print('T-Statisticnya adalah :',t_stat)
print('P-valuenya adalah :',p_val)

# langkah selanjutkan kita tentukan tingkat signifikansi (sisa 95% confidence percent)
alpha = 0.05

if p_val < alpha:
    print('Hipotesis Null ditolak. Harga barang di Jabodetabek tidak sama dengan harga di luar Jabodetabek.')
else:
    print('Tidak cukup bukti untuk menolak Hipotesis Null. Harga barang di Jabodetabek sama dengan harga di luar Jabodetabek.')

T-Statisticnya adalah : 0.3728079972067453
P-valuenya adalah : 0.7102304071426224
Tidak cukup bukti untuk menolak Hipotesis Null. Harga barang di Jabodetabek sama dengan harga di luar Jabodetabek.


In [196]:
# membandingkan apakah harga produk mempengaruhi rating/kesukaan pembeli kepada produk
corr_r, pval_p = stats.pearsonr(df['Harga_Produk'], df['Rating_Produk'])
corr_rho, pval_s = stats.spearmanr(df['Harga_Produk'], df['Rating_Produk'])
corr_tau, pval_k = stats.kendalltau(df['Harga_Produk'], df['Rating_Produk'])

print(f"correlation r : {corr_r:.2f},  p-value : {pval_p}")
print(f"correlation rho : {corr_rho:.2f},  p-value : {pval_s}")
print(f"correlation tau : {corr_tau:.2f},  p-value : {pval_k}")

correlation r : 0.01,  p-value : 0.9263174835028429
correlation rho : 0.02,  p-value : 0.8694817451016685
correlation tau : 0.01,  p-value : 0.8588858057699518


dari syntax diatas dapat kita simpulkan bahwa semakin tinggi harga produk maka akan semakin tinggi pula ratingnya

# E. Conclusion

Pada program ini kita dapat mengetahui dari list tiap kolomnya yang bisa kita analisa mean, median, skew, dll untuk dapat kita hitungkan dengan nilai lower dan uppernya untuk dapat melihat hasil dari perhitungan terendah dan tertinggi dalam penjualan produk seblak, selain itu kita juga dapat mengetahui perbandingan harga tiap produk seblak apakah sama atau tidak dari daerah jabodetabek maupun di luar jabodetabek, dan kita dapat menyimpulkan bahwa semakin tinggi harga produk makan akan semakin tinggi juga rating dan tingkat kepuasan pelanggan yang membeli seblak.   